In [6]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.PropertyMol import PropertyMol
Chem.SetDefaultPickleProperties(Chem.PropertyPickleOptions.AllProps)
import pickle
import sys
from pathlib import Path

In [7]:
sys.path.append('/home/paula/Masterarbeit/KinaseFocusedFragmentLibrary/kinase_focused_fragment_library/analysis/ligand_analysis')
sys.path.append('/home/paula/Masterarbeit/KinaseFocusedFragmentLibrary/kinase_focused_fragment_library/recombination')
from construct_ligand import construct_ligand, read_fragment_library
from novelty import read_scaffolds

In [8]:
def pickle_loader(pickle_file):

    try:
        while True:
            yield pickle.load(pickle_file)
    except EOFError:
        pass

In [9]:
combinatorial_library_folder = Path('/home/paula/Masterarbeit/CombinatorialLibrary/')
file_name = combinatorial_library_folder / 'combinatorial_library.pickle'

In [12]:
subpockets = ['AP', 'FP', 'SE', 'GA', 'B1', 'B2']
data = read_fragment_library(Path('/home/paula/Masterarbeit/FragmentLibrary'), subpockets)

Number of fragments in AP : 2559
Number of fragments in FP : 1865
Number of fragments in SE : 1560
Number of fragments in GA : 1023
Number of fragments in B1 : 98
Number of fragments in B2 : 92


In [13]:
# analyze subpocket arrangements
conn_count = {}
count = 0

with open(file_name, 'rb') as pickle_file:
    
    for ligand in pickle_loader(pickle_file):
        count += 1
        if count % 1000000 == 0:
            print(count)
        
        mol = Chem.RemoveHs(construct_ligand(ligand.meta, data)[0])
        
        for atom in mol.GetAtoms():
            subpocket = atom.GetProp('subpocket')
            for neigh in atom.GetNeighbors():
                n_subpocket = neigh.GetProp('subpocket') 
                if n_subpocket != subpocket:
                    conn = frozenset((subpocket, n_subpocket))
                    if conn in conn_count:
                        conn_count[conn] += 1
                    else:
                        conn_count[conn] = 1

In [14]:
conn_count

{frozenset({'AP', 'SE'}): 1248,
 frozenset({'AP', 'FP'}): 1006,
 frozenset({'AP', 'GA'}): 1118,
 frozenset({'FP', 'SE'}): 80,
 frozenset({'FP', 'GA'}): 144,
 frozenset({'B1', 'GA'}): 132}